In [1]:
import pandas as pd

# Load the student data
file_path = 'https://raw.githubusercontent.com/ifiecas/student_data/main/student_data4.csv'  # Replace with the correct path to your CSV file
students_df = pd.read_csv(file_path)

# Load the course data
file_path = 'https://raw.githubusercontent.com/ifiecas/courseplan/main/bmbu-2024_ver7.csv'  # Replace with the correct path to your CSV file
course_df = pd.read_csv(file_path)



In [ ]:

def student_login(student_id, password, students_df):
    # Ensure that both inputs are stripped of leading/trailing spaces and converted to lowercase
    student_id = student_id.strip().lower()
    password = password.strip()
    
    # Convert the relevant columns in the DataFrame to strings, strip any extra spaces, and lowercase for case-insensitive comparison
    students_df['st_id'] = students_df['st_id'].astype(str).str.strip().str.lower()
    students_df['password'] = students_df['password'].astype(str).str.strip()
    
    # Check if the student ID and password match
    student_data = students_df[(students_df['st_id'] == student_id) & 
                               (students_df['password'] == password)]
    
    if not student_data.empty:
        print(f"\nWelcome, {student_data['first_name'].values[0]}!")
        return student_data.iloc[0]  # Return as a Series for easier access
    else:
        print("\nInvalid student ID or password. Please try again.")
        return None

def display_profile(student_data):
    print("\n--- Student Profile ---")
    print(f"Name            : {student_data['first_name']}")
    print(f"Student ID      : {student_data['st_id']}")
    print(f"Gender          : {student_data['gender']}")
    
    print("\nCompleted Units:")
    completed_units = student_data['completed_units'].split(',')
    total_credits = 0
    
    print("\nUnit Code, Semester Completed, Credits Earned")
    for unit in completed_units:
        unit = unit.strip()
        
        # Extract Unit Code, Semester, and Credits using parsing
        if '(' in unit and ')' in unit:
            unit_code = unit.split(' ')[0]
            details = unit.split('(')[1].replace(')', '')
            semester = details.split(',')[0].strip()
            credits = int(details.split('Credits ')[1].strip())
            total_credits += credits
            print(f"  - {unit_code}, {semester}, {credits} Credits")
        else:
            print(f"  - {unit} (Details not found in expected format)")
    
    print(f"\nTotal Credits Earned: {total_credits}")
    print("-----------------------\n")




def view_enrollment_history(student_data):
    print("\n--- Enrollment History ---")
    completed_units = student_data['completed_units'].split(',')
    print("Completed Units:")
    for unit in completed_units:
        print(f"- {unit.strip()}")
    print("-------------------------\n")

def check_eligibility(student_data, course_code, course_df):
    # Get the course details
    course_data = course_df[course_df['UNIT_CODE'] == course_code]
    
    if course_data.empty:
        return False, f"Course {course_code} does not exist."
    
    # Retrieve course details
    unit_type = course_data['UNIT_TYPE'].values[0]
    credit = course_data['CREDIT'].values[0]
    unit_title = course_data['UNIT_TITLE'].values[0]
    prereq_1 = course_data['PREREQUISITES_1'].values[0]
    prereq_2 = course_data['PREREQUISITES_2'].values[0]
    
    # Check gender-specific restriction for WOM1000
    if course_code == 'WOM1000' and student_data['gender'].lower() != 'female':
        return False, f"Course {course_code} - {unit_title} is only available for female students."
    
    # Create a list of all prerequisites to check
    all_prereqs = []
    
    if pd.notna(prereq_1) and prereq_1.strip():
        all_prereqs.append(prereq_1.strip())
    if pd.notna(prereq_2) and prereq_2.strip():
        all_prereqs.append(prereq_2.strip())
    
    # Check if the student has completed the prerequisite courses
    completed_courses = student_data['completed_units'].split(',')

    # Eligibility: Student must have completed all prerequisites
    if all(prereq in completed_courses for prereq in all_prereqs):
        return True, f"Eligible for {course_code} - {unit_title} ({unit_type}, {credit} credits)."
    else:
        return False, f"Not eligible for {course_code} - {unit_title} ({unit_type}, {credit} credits). Prerequisites not met."

def enroll_student(student_data, course_codes, course_df):
    enrolled_courses = []
    
    for course_code in course_codes:
        eligible, message = check_eligibility(student_data, course_code, course_df)
        if eligible:
            enrolled_courses.append(course_code)
            print(f"{message}")
        else:
            print(message)
    
    if len(enrolled_courses) > 0:
        print("\nYou have successfully enrolled in the following unit(s):")
        for idx, course_code in enumerate(enrolled_courses, start=1):
            course_data = course_df[course_df['UNIT_CODE'] == course_code]
            unit_title = course_data['UNIT_TITLE'].values[0]
            unit_type = course_data['UNIT_TYPE'].values[0]
            credit = course_data['CREDIT'].values[0]
            print(f"  {idx}. {course_code} - {unit_title} ({unit_type}, {credit} credits)")
        
        # Ask if the student wants to go back to the main menu
        back_to_menu = input("\nWould you like to go back to the main menu? (yes/no): ").strip().lower()
        if back_to_menu == 'yes':
            return True  # Return to main menu
        else:
            print("\nThank you for using the enrollment system. Goodbye!")
            return False  # Exit the program
    else:
        print(f"Student {student_data['st_id']} is not eligible for any of the selected courses.\n")
        return True  # Return to main menu

def provide_feedback():
    feedback = input("\nPlease provide your feedback: ")
    print("\nThank you for your feedback!\n")

def main():
    student_id = input("Enter your student ID: ")
    password = input("Enter your password: ")
    
    # Log in the student
    student_profile = student_login(student_id, password, students_df)
    
    if student_profile is not None:
        display_profile(student_profile)
        
        while True:
            print("What would you like to do?")
            print("1. View My Profile")
            print("2. View Enrollment History")
            print("3. Enroll in a Course")
            print("4. Provide Feedback")
            print("5. Logout")
            choice = input("Enter the number of your choice: ").strip()
            
            if choice == '1':
                display_profile(student_profile)
            elif choice == '2':
                view_enrollment_history(student_profile)
            elif choice == '3':
                course_1 = input("Enter the first course code you want to enroll in: ").strip()
                course_2 = input("Enter the second course code you want to enroll in (if any): ").strip()
                
                course_codes = [course_1]
                if course_2:
                    course_codes.append(course_2)
                
                continue_menu = enroll_student(student_profile, course_codes, course_df)
                if not continue_menu:
                    break
            elif choice == '4':
                provide_feedback()
            elif choice == '5':
                print("\nLogging out...\n")
                break
            else:
                print("\nInvalid choice. Please try again.\n")

# Ensure that all data is loaded before running main
if __name__ == "__main__":
    main()


Enter your student ID:  95841
Enter your password:  3VEuK



Welcome, Fernando!

--- Student Profile ---
Name            : Fernando
Student ID      : 95841
Gender          : Male

Completed Units:

Unit Code, Semester Completed, Credits Earned
  - BCO7000 (Sem 3 (Details not found in expected format)
  - Credits 12) (Details not found in expected format)
  - BCO7006 (Sem 3 (Details not found in expected format)
  - Credits 12) (Details not found in expected format)
  - BEO6000 (Sem 3 (Details not found in expected format)
  - Credits 12) (Details not found in expected format)
  - BCO7004 (Sem 4 (Details not found in expected format)
  - Credits 12) (Details not found in expected format)

Total Credits Earned: 0
-----------------------

What would you like to do?
1. View My Profile
2. View Enrollment History
3. Enroll in a Course
4. Provide Feedback
5. Logout
